In [1]:
import logging
import os
from pprint import pprint

import dask.dataframe as dd
import helpers
import numpy as np
import pandas as pd
import scipy.stats
from cloudpathlib import AnyPath as Path

In [2]:
handler = logging.StreamHandler()
formatter = logging.Formatter(
    "%(asctime)s %(process)d/%(threadName)s %(name)s %(levelname)s\n%(message)s"
)
handler.setFormatter(formatter)
logging.getLogger().handlers = [handler]

In [3]:
logger = logging.getLogger(__name__)
logger.setLevel("DEBUG")

In [5]:
uri_real_bulk_rnaseq = "gs://liulab/data/pseudobulk_optimization/3_with_tcga_qc/mixtures_real_tcga_skcm/tpm.parquet"
real_bulk_rnaseq = dd.read_parquet(uri_real_bulk_rnaseq, engine="pyarrow")
logger.debug(real_bulk_rnaseq.dtypes)

2022-07-04 16:32:48,047 32429/MainThread __main__ DEBUG
gene_symbol        category
aliquot_barcode    category
tpm                 float64
dtype: object


In [6]:
uri_pseudobulk_rnaseq = (
    "gs://liulab/data/pseudobulk_optimization/3_with_tcga_qc/mixtures"
)
# !gsutil ls -lhR {uri_pseudobulks} | grep data.parquet | head
pseudobulk_rnaseq = (
    # all this stuff
    dd.read_parquet(uri_pseudobulk_rnaseq, engine="pyarrow")
    .replace({"malignant_from_one_sample": {"True": True, "False": False}})
    .astype({"n_cells": "uint8", "malignant_from_one_sample": "bool"})
)
logger.debug(pseudobulk_rnaseq.dtypes)

2022-07-04 16:32:48,512 32429/MainThread __main__ DEBUG
gene_symbol                           category
tcga_aliquot_barcode_for_fractions    category
tpm                                    float64
n_cells                                  uint8
malignant_from_one_sample                 bool
dtype: object


In [8]:
pseudobulk_rnaseq.query("n_cells == 5 and malignant_from_one_sample == True").compute()

/opt/conda/envs/deconv/lib/python3.10/site-packages/dask/utils.py:1053: FutureWarning: The `inplace` parameter in pandas.Categorical.rename_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  return getattr(__obj, self.method)(*args, **kwargs)


,gene_symbol,tcga_aliquot_barcode_for_fractions,tpm,n_cells,malignant_from_one_sample
0,A1BG,TCGA-3N-A9WB-06A-11R-A38C-07,106.644738,5,True
1,A1BG,TCGA-3N-A9WC-06A-11R-A38C-07,72.082156,5,True
2,A1BG,TCGA-3N-A9WD-06A-11R-A38C-07,48.453905,5,True
3,A1BG,TCGA-BF-AAP0-06A-11R-A39D-07,29.958744,5,True
4,A1BG,TCGA-D3-A1Q1-06A-21R-A18T-07,38.982455,5,True
...,...,...,...,...,...
5911179,ZZZ3,TCGA-YG-AA3O-06A-11R-A38C-07,51.707960,5,True
5911180,ZZZ3,TCGA-YG-AA3P-06A-11R-A38C-07,52.941551,5,True
5911181,ZZZ3,TCGA-Z2-A8RT-06A-11R-A37K-07,34.450564,5,True
5911182,ZZZ3,TCGA-Z2-AA3S-06A-11R-A39D-07,190.622405,5,True


In [10]:
Out[8]["tcga_aliquot_barcode_for_fractions"].value_counts()

TCGA-3N-A9WB-06A-11R-A38C-07    16063
TCGA-ER-A2NF-06A-11R-A18T-07    16063
TCGA-FR-A3YN-06A-11R-A239-07    16063
TCGA-ER-A42L-06A-11R-A24X-07    16063
TCGA-ER-A42K-06A-11R-A24X-07    16063
                                ...  
TCGA-EE-A17X-06A-11R-A18S-07    16063
TCGA-EB-A6L9-06A-11R-A32P-07    16063
TCGA-EB-A5VV-06A-11R-A32P-07    16063
TCGA-EB-A5UN-06A-11R-A311-07    16063
TCGA-Z2-AA3V-06A-11R-A39D-07    16063
Name: tcga_aliquot_barcode_for_fractions, Length: 368, dtype: int64